# DICOM FlowMRI Reader - analysis of DICOM files and conversion to hpc-predict-io HDF5-format

In [ ]:
# Notebook parameters - to be moved into papermill parameters
import os
hpc_predict_code_root = "/src/hpc-predict"
hpc_predict_data_root = "/hpc-predict-data" # os.path.join(os.environ['HOME'], "src/hpc-predict/hpc-predict/data/v1")

mri_samples = [3,]
tar_files = True
mri_data_root = "input_data/original/mri/MRT Daten Bern tar"
output_root   = "input_data/preprocessed/mri/MRT Daten Bern tar"

In [ ]:
# Turn mri_samples into a list if it isn't one yet (papermill)
if type(mri_samples) is not list:
    mri_samples = [mri_samples]
    
# Bash version of list
mri_samples_bash = ' '.join([str(sample) for sample in mri_samples])
tar_files_bash = '--tar' if tar_files else ''
mri_samples_bash

In [ ]:
%env hpc_predict_code_root = {hpc_predict_code_root}
%env hpc_predict_data_root = {hpc_predict_data_root}
%env mri_data_root = {mri_data_root}
%env mri_samples = {mri_samples_bash}
%env tar_files = {tar_files_bash}
%env output_root = {output_root}

# DICOM header extraction into Pandas DataFrame and overview

This script assumes that you have the Freiburg dataset available, otherwise first run `data/fetch_scripts/fetch_freiburg_to_tar.sh data/tmp data/v1`.

To run the following steps, the DICOM headers must previously be converted to pandas tables, i.e.

In [ ]:
%%bash
set -x
python3  "${hpc_predict_code_root}"/hpc-predict-io/mri_datasource/freiburg/convert_dicom_headers_to_pandas_df.py --hpc-predict-data-root "${hpc_predict_data_root}" --mri-data-root "${mri_data_root}" --mri-samples ${mri_samples} ${tar_files} --output-root "${output_root}"
set +x

In [ ]:
import sys
import tarfile
import json
import pydicom
import pandas as pd
import numpy as np
from pprint import pprint
from mr_io import FlowMRI

In [ ]:
# Possibly duplicate with code in convert_dicom_headers_to_pandas_df

if os.path.exists(os.path.join(hpc_predict_data_root, 'encrypt')):
    hpc_predict_data_root = os.path.join(hpc_predict_data_root, 'decrypt')
    assert os.path.exists(hpc_predict_data_root)

mri_data_root = os.path.join(hpc_predict_data_root, mri_data_root)
assert os.path.exists(mri_data_root)

for sample in mri_samples:
    assert os.path.exists(os.path.join(mri_data_root, str(sample) + ('.tar' if tar_files else '/') ))

if tar_files:
    mri_sample_tar_paths = [os.path.relpath(os.path.join(mri_data_root, str(sample) + '.tar'), start=hpc_predict_data_root) for sample in mri_samples]
    

output_root = os.path.join(hpc_predict_data_root, output_root)
assert os.path.exists(output_root)

### Load pickled DataFrames and metainformation

In [ ]:
with open(os.path.join(hpc_predict_data_root, mri_data_root,"venc.json"), 'r') as f:
    venc = json.load(f)
venc

In [ ]:
pkls = sorted([os.path.join(output_root, str(sample), "dicom_header_df.pkl") for sample in mri_samples], key=lambda x: int(os.path.basename(os.path.dirname(x))) ) # os.path.basename(x).split('.')[0]
pkls

In [ ]:
# Read pandas DataFrame that contains pydicom DataElement objects in individual entries 
df = pd.concat([pd.read_pickle(pkl) for pkl in pkls],axis=0)

In [ ]:
# A pydicom Dataelement: The value is held in the _value attribute, 
# often in "semi-serialized" representation with the VR describing the DICOM type  
vars(df.iloc[0,0])

### Use names instead of DICOM tags as column labels

In [ ]:
## Compute the DICOM tag -> name mapping to rename columns
tag_names = df.apply( lambda c: c.dropna().apply(lambda x: x.name).unique(), axis=0); assert tag_names.shape[0] == 1; tag_names = tag_names.loc[0] 
#tag_names.to_dict()

In [ ]:
df_renamed = df.rename(columns=tag_names.to_dict())

# display only values
pd.set_option('display.max_columns', 20)
df_renamed.applymap(lambda x: x.value if pd.notnull(x) else x )

### Determine range of DICOM tags (via unique rows, grouped by patient ID > study UID > series UID)

In [ ]:
# Determine types of DataElement.value entries
from collections import namedtuple, OrderedDict
from types import SimpleNamespace
import json

pd.set_option('display.max_columns', None)
pd.concat([pd.Series(name=name, data=col.dropna().unique()) for (name, col) in df_renamed.applymap(lambda x: (x.VR, type(x.value)) if pd.notnull(x) else x).iteritems()], axis=1)

In [ ]:
# Extract native Python datatypes (built-in/standard library) from pydicom DataElements
def get_native_from_pydicom(val):
    from pydicom.valuerep import DA, DT, TM, DSfloat, DSdecimal, IS, PersonName
    from pydicom.uid import UID
    from pydicom.multival import MultiValue
    from datetime import (date, datetime, time, timedelta, timezone)
    from decimal import Decimal

    if isinstance(val, DA):
        return date(val.year, val.month, val.day)
    elif isinstance(val, DT):
        return datetime(val.year, val.month, val.day,
                        val.hour, val.minute, val.second,
                        val.microsecond, val.tzinfo)
    elif isinstance(val, TM):
        return time(val.hour, val.minute, val.second,
                    val.microsecond)
    elif isinstance(val, DSfloat):
        return float(val)
    elif isinstance(val, DSdecimal):
        return Decimal(val)
    elif isinstance(val, IS):
        return int(val)
    elif isinstance(val, MultiValue):
        return tuple([get_native_from_pydicom(el) for el in val])
    elif isinstance(val, UID):
        return str(val)
    elif isinstance(val, (PersonName, str, int, float)):
        return val
    elif isinstance(val, list):
        return tuple(val)
    else:
        return val        

def unpack_pydicom_value(value):
    if hasattr(type(value),'__module__') and type(value).__module__.startswith('pydicom'):
        return get_native_from_pydicom(value)
    elif isinstance(value, list):
        return tuple([unpack_pydicom_value(v) for v in value])
    elif isinstance(value,OrderedDict):
        return tuple( [unpack_pydicom_value(v.value) for v in value.values()] )
    elif isinstance(value, dict):
        return tuple(value.items())
    else:
        return value
    
def agg_unique(group):
    labels = []
    row = []
    for (name, col) in group[group.columns[:]].iteritems():
        labels.append(name)
        row.append(col.dropna().unique())

    return pd.DataFrame([row], columns=labels)

In [ ]:
df_native_renamed = df_renamed.applymap(lambda x: unpack_pydicom_value(x.value) if pd.notnull(x) else x)

## Group-by patient/study/series ID

In [ ]:
group_by_series_keys = ['FileCollectionID',
                        'Instance Creation Date',
                        'Patient ID',
                        'Study Instance UID',
                        'Sequence Name',
                        'Series Instance UID',
                        'Image Type']

df_native_grouped = df_native_renamed.groupby(group_by_series_keys)
df_native_grouped_unique = df_native_grouped.apply(lambda x: agg_unique(x))  

## All tags with <= 1 unique values per series in any series

In [ ]:
df_native_grouped_unique_singleton_cols = df_native_grouped_unique.applymap(lambda x: len(x) <= 1).all()
df_native_grouped_unique.drop(
    df_native_grouped_unique.columns[~df_native_grouped_unique_singleton_cols], axis=1).applymap(lambda x: x[0] if len(x) == 1 else x)

## ...and number of unique values

In [ ]:
df_native_grouped_unique.drop(
    df_native_grouped_unique.columns[df_native_grouped_unique_singleton_cols],axis=1).applymap(lambda x: len(x))

## All tags with more than a single unique values in at least one series

In [ ]:
df_native_grouped_unique.drop(
    df_native_grouped_unique.columns[df_native_grouped_unique_singleton_cols],axis=1).applymap(
    lambda x: x[0] if len(x) == 1 else x).drop(['FilePath'], axis=1)

## DICOM header analysis

### Analysis: Referenced Image Sequence tag

In [ ]:
example_referenced_uid = df_native_renamed['Referenced Image Sequence'].iloc[0][0][1]

# Check if we can find the referenced image sequence
print("The string length of a Referenced Image Sequence UID is {}: {}.".format(len(example_referenced_uid), example_referenced_uid))
        
def get_uid_prefix(uid):
    return '.'.join(uid.split('.')[:-1])
example_referenced_uid_prefix = get_uid_prefix(example_referenced_uid)
print("Example referenced UID prefix is {}".format(example_referenced_uid_prefix))

print("Share of SOP Instance UIDs with same prefix up to and after last point: {}.".format(', '.join([
    str(100.*
    (df_native_renamed['SOP Instance UID'].apply(lambda x: x[:prefix_len] ) == example_referenced_uid[:prefix_len] ).sum()/
    df_native_renamed.shape[0]) + " %" for prefix_len in range(len(example_referenced_uid_prefix),len(example_referenced_uid_prefix)+10)]) ) )

Conclusion: Referenced Image Sequence Tag identical before last point, afterwards ~random - the prefix 1.2.40.0.13.1 is taken from the dcm4che DICOM implementation, the remainder is a randomly generated UID.

### Analysis of time (Acquisition time, Content time, Trigger time and Nominal interval)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
import datetime  
from types import SimpleNamespace

In [ ]:
group_by_series_title = "Group by series UID plot (" + \
                        " / ".join([group_by_series_keys[0],
                                    group_by_series_keys[1],
                                    group_by_series_keys[4],
                                    group_by_series_keys[6],
                                    'Nominal Interval',
                                    'Cardiac Number of Images',
                                    '[P-index]'
                                  ]) + \
                        ")"

def legend_unique_values(ls):
    return str(ls[0]) if len(ls)==1 else "{}..{}".format(ls.min(), ls.max()) 

def group_by_series_legend(name, group, j=None):
    nominal_intervals = group['Nominal Interval'].unique() #.apply(lambda x: x.original_string).unique()
    cardiac_number_of_images = group['Cardiac Number of Images'].unique() #.apply(lambda x: x.original_string).unique()
    return ", ".join([
            str(name[0]),
            str(name[1]),
            name[4],
            str(name[6]), 
            "NomInt {} ms".format(legend_unique_values(nominal_intervals)),
            "CardImgs {}".format(legend_unique_values(cardiac_number_of_images))
            ]) + (", P-{}".format(j) if j is not None else "")    

In [ ]:
#from inspect import signature

# groups = df.\
#     applymap(lambda x: unpack_pydicom_value(x.value) if pd.notnull(x) else x).\
#     groupby(group_by_series_keys)

def make_group_by_plot(groups, x_series, x_label, y_series, y_label):    
    
    ngroups = len(groups.groups)
    ncols = 3
    nrows = (ngroups+3-1)//ncols    
    
    fig, ax = plt.subplots(nrows=nrows,ncols=ncols, sharex=True, figsize=(12*ncols,8*nrows))
    ax = ax.flatten()
    
    for i, (name, group) in enumerate(groups): 

        if 'P' not in group['Image Type'].iloc[0]:
            ax[i].plot(x_series(group), 
                       y_series(group), 
                       'o',  ms=6, linewidth=1, 
                       label=group_by_series_legend(name,group))
        else:        
            for j in range(3):
                select_subgroup = group['Instance Number'].apply(lambda x: j*group.shape[0]//3 < x and x <= (j+1)*group.shape[0]//3)            
                ax[i].plot(x_series(group[select_subgroup]), 
                           y_series(group[select_subgroup]),
                           'o',  ms=6, linewidth=1, 
                           label=group_by_series_legend(name,group,j))

    for a in ax[:ngroups]: 
        a.set_xlabel(x_label)
        a.set_ylabel(y_label)

        a.legend(bbox_to_anchor=(-0.3, 1.04), loc="lower left")
        a.grid()
    fig.suptitle(group_by_series_title)
    fig.tight_layout(pad=3.)

### Acquisition Time vs. Trigger Time

In [ ]:
def acquisition_datetime(df):
    return df.apply(
        lambda x: datetime.datetime(x['Series Date'].year, 
                                    x['Series Date'].month, 
                                    x['Series Date'].day, 
                                    x['Acquisition Time'].hour, 
                                    x['Acquisition Time'].minute,
                                    x['Acquisition Time'].second, 
                                    x['Acquisition Time'].microsecond, 
                                    x['Acquisition Time'].tzinfo ), axis=1)

def acqusition_datetime_offset(df):
    acquisition_datetime_series = acquisition_datetime(df)
    return acquisition_datetime_series - acquisition_datetime_series.min()


group_x_series = lambda group: group['Trigger Time']
x_label = 'Trigger Time [ms]'
group_y_series = lambda group: acqusition_datetime_offset(group).apply(lambda x: x.total_seconds()*1e3)+group['Trigger Time'].min()
y_label = 'Acquisition Time (offset to min) [ms]'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

It seems that the Trigger Time and Acquisition time coincide up to an offset for Acquisition time. Also, they do not cover the entire heart cycle but only a bit more than the first half. This could be the case since it may be a prospective hear MRI as explained [here](http://mriquestions.com/gating-parameters.html).

In [ ]:
# Trigger time vs. nominal interval
pd.concat([df_native_grouped_unique[['Trigger Time']].applymap(lambda x: np.max(x)), df_native_grouped_unique[['Nominal Interval']]], axis=1)

### Content Time vs. Trigger Time

In [ ]:
def content_datetime(df):
    return df.apply(
        lambda x: datetime.datetime(x['Series Date'].year, 
                                    x['Series Date'].month, 
                                    x['Series Date'].day, 
                                    x['Content Time'].hour, 
                                    x['Content Time'].minute,
                                    x['Content Time'].second, 
                                    x['Content Time'].microsecond, 
                                    x['Content Time'].tzinfo ), axis=1)

def content_datetime_offset(df):
    content_datetime_series = content_datetime(df)
    return content_datetime_series - content_datetime_series.min()

group_x_series = lambda group: group['Trigger Time']
x_label = 'Trigger Time [ms]'
group_y_series = lambda group: content_datetime_offset(group).apply(lambda x: x.total_seconds()*1e3)+group['Trigger Time'].min()
y_label = 'Content Time (offset to min) [ms]'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

Seems that the Content Time refers to the time when the MRI signal was recorded - the entire sequence is spread over more than 40 seconds. No structure apparent from these plots

### Trigger Time vs. Instance Number

In [ ]:
group_x_series = lambda group: group['Instance Number'] 
x_label = 'Instance Number'
group_y_series = lambda group:  group['Trigger Time'] 
y_label = 'Trigger Time [ms]'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

These plots clearly suggest that the P-series contain 3 sub-series that are sorted according to instance number (the visible plateau is due to multiple slices per fixed time being recorded one after another). 

### Content Time vs. Instance Number

In [ ]:
group_x_series = lambda group: group['Instance Number']
x_label = 'Instance Number'
group_y_series = lambda group: content_datetime_offset(group).apply(lambda x: x.total_seconds()*1e3)+group['Trigger Time'].min()
y_label = 'Content Time (offset to min) [ms]'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)


No correlation between content time and instance number.

### Instance Creation Time vs. Instance Number

In [ ]:
def instance_creation_datetime(df):
    return df.apply(
        lambda x: datetime.datetime(x['Series Date'].year, 
                                    x['Series Date'].month, 
                                    x['Series Date'].day, 
                                    x['Instance Creation Time'].hour, 
                                    x['Instance Creation Time'].minute,
                                    x['Instance Creation Time'].second, 
                                    x['Instance Creation Time'].microsecond, 
                                    x['Instance Creation Time'].tzinfo ), axis=1)

def instance_creation_datetime_offset(df):
    content_datetime_series = content_datetime(df)
    return content_datetime_series - content_datetime_series.min()

group_x_series = lambda group: group['Instance Number']
x_label = 'Instance Number'
group_y_series = lambda group: instance_creation_datetime_offset(group).apply(lambda x: x.total_seconds()*1e3)+group['Trigger Time'].min()
y_label = 'Instance Creation Time (offset to min) [ms]'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

### SOP Instance UID vs. Instance Number

In [ ]:
group_x_series = lambda group: group['Instance Number'] 
x_label = 'Instance Number'
group_y_series = lambda group:  group['SOP Instance UID'].apply(lambda x: int(x.split('.')[-1]))
y_label = 'SOP Instance UID'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)


In [ ]:
# Unique SOP Instance UID prefixes
df_native_renamed['SOP Instance UID'].apply(lambda x: get_uid_prefix(x)).unique()

The SOP Instance UID seems to be an Implementation label for dcm4che followed by a randomly generated UID with no correlation with Instance Number.

### Slice Location vs. Instance Number

In [ ]:
group_x_series = lambda group: group['Instance Number']
x_label = 'Instance Number'
group_y_series = lambda group: group['Slice Location']
y_label = 'Slice Location'
make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

### Slice Location (col x row coord sys) - Slice Location (DICOM) vs. Instance Number

In [ ]:
group_x_series = lambda group: group['Instance Number'] #.apply(lambda x: float(x.original_string))
x_label = 'Instance Number'

def slice_location_image_coord_sys(group):
    seq_origin = np.array(group.loc[group['Instance Number'].idxmin(),'Image Position (Patient)'])    
    
    orientation = group['Image Orientation (Patient)'].unique()
    assert len(orientation) == 1

    row_unit = np.array(orientation[0][:3])
    col_unit = np.array(orientation[0][3:])
    ort_unit = np.cross(col_unit, row_unit)

    return group['Image Position (Patient)'].apply(lambda x: np.dot(ort_unit, np.array(x)-seq_origin) )

def slice_location_diff(group):
    return slice_location_image_coord_sys(group) - group['Slice Location'].apply(lambda x: np.array(x))

group_y_series = slice_location_diff
y_label = 'Slice Location (col x row) - Slice Location (DICOM)'

make_group_by_plot(df_native_grouped, group_x_series, x_label, group_y_series, y_label)

The coordinate system of the DICOM images produces the same alignment (with "Slice Location" as z-axis) when taking the column direction as the primary and row-direction as the secondary axis (i.e. x runs along the columns, y along the rows).

### Check orthogonality of image planes and relative image offset vectors

In [ ]:
# for i, (name, group) in enumerate(pd.concat([df_renamed, instance_creation_datetime], axis=1).\
#     applymap(lambda x: unpack_pydicom_value(x.value) if pd.notnull(x) else x).\
#     groupby(['FileCollectionID', 'Instance Creation Date', 'Patient ID', 'Study Instance UID', 'Series Instance UID', 'Image Type', 'Sequence Name'])): # group['Instance Creation Date'].iloc[0]

def print_if_above_numeric_tol(name, value, tol=1e-11):
    if value > tol:
        print("    FAIL: {} = {} exceeds numeric tolerance ".format(name, value))
        raise RuntimeError()
    else:
        print("    OK:   {} = {} ".format(name, value))
            

for i, (name, group) in enumerate(df_native_grouped):

    orientation = group['Image Orientation (Patient)'].unique()
    assert len(orientation) == 1

    row_unit = np.array(orientation[0][:3])
    col_unit = np.array(orientation[0][3:])
    ort_unit = np.cross(col_unit, row_unit)
    

    if 'P' not in group['Image Type'].iloc[0]:

        print(name[0],name[1],name[4], name[6])

        positions = group.sort_values('Instance Number')['Image Position (Patient)'].apply(lambda x: np.array(x))    
        #pprint((positions.iloc[1:] - positions[:-1]).apply(lambda x: np.linalg.norm(x)).max())
        if len(positions) > 1:
            print_if_above_numeric_tol("max(dot(col_unit, patient_position_offset))", np.abs(np.dot( np.vstack(positions.to_numpy()[1:] - positions.to_numpy()[:-1]), col_unit) ).max() )
            print_if_above_numeric_tol("max(dot(row_unit, patient_position_offset))", np.abs(np.dot( np.vstack(positions.to_numpy()[1:] - positions.to_numpy()[:-1]), row_unit) ).max() )

    else:
        for j in range(3):

            print(name[0], name[1], name[4], name[6], j)

            select_subgroup = group['Instance Number'].apply(lambda x: j*group.shape[0]//3 < x and x <= (j+1)*group.shape[0]//3)            

            positions = group[select_subgroup].sort_values('Instance Number')['Image Position (Patient)'].apply(lambda x: np.array(x))
            if len(positions) > 1:
                print_if_above_numeric_tol("max(dot(col_unit, patient_position_offset))", np.abs(np.dot( np.vstack(positions.to_numpy()[1:] - positions.to_numpy()[:-1]), col_unit) ).max() )
                print_if_above_numeric_tol("max(dot(row_unit, patient_position_offset))", np.abs(np.dot( np.vstack(positions.to_numpy()[1:] - positions.to_numpy()[:-1]), row_unit) ).max() )


Image planes are orthogonal to Image Position (Patient) offset vector (note that the calculation wraps around time slice edges, i.e. it computes offsets between last and first Image Position (Patient), which, however, does not affect the conclusion).

In [ ]:
#TODO:
# - Check that Cardiac number of images coincides with number of times (trigger/acquisition)
# - number of slice locations coincides with number of image position patients
# - and that both together with Rows and Columns explain the number of instances
# - statistics over unique values per series and conversion to HPC-PREDICT-IO/create MRI writer 

# Conversion to hpc-predict-io HDF5-format

In [ ]:
df_native_grouped_unique[['Image Type', 'FilePath_4']]

In [ ]:
print("Found sequence names {} (only processing fl3d1).".format(df_native_renamed['Sequence Name'].unique()))

group_by_series_keys = ['FileCollectionID', 'Instance Creation Date', 'Patient ID', 'Study Instance UID', 'Sequence Name'] #, 'Series Instance UID', 'Image Type']
df_flash_grouped = df_native_renamed[df_native_renamed['Sequence Name'] == 'fl3d1'].groupby(group_by_series_keys)

In [ ]:
if tar_files:
    tar_files_dict = {
        tar_path: tarfile.open(os.path.join(hpc_predict_data_root, tar_path)) 
        for tar_path in mri_sample_tar_paths
    }
    
def dcmread(file_path):
    if not tar_files:
        return pydicom.dcmread(os.path.join(hpc_predict_data_root, file_path))
    else:
#         tar = tarfile.open(os.path.join(hpc_predict_data_root, file_path.split(':')[0]))
        tar = tar_files_dict[file_path.split(':')[0]]
        dcmfile = tar.extractfile(file_path.split(':')[1])
        return pydicom.filereader.dcmread(dcmfile)

In [ ]:
# sequence handler for fl3d1, that for Nav can be null

flow_mris = []

def extract_unique_value(group, col):
    value = group[col].unique()
    assert len(value) == 1; 
    return value[0]

for (name, group) in df_flash_grouped:
    print(name,flush=True)
    
    file_path = dict(M=set(), P=set(), M_files=[], P_files=[])
    magnitude = None
    phase = []
    
    for (series_name, series_group) in group.groupby(['Series Instance UID', 'Image Type']):
        if 'M' in series_name[1]:
            magnitude = series_group.sort_values('Instance Number')
            file_path['M'].add(extract_unique_value(series_group, 'FilePath_4'))
            file_path['M_files'].append(list(magnitude['FilePath'].values))
        elif 'P' in series_name[1]:
            phase_group = series_group.sort_values('Instance Number')
            phase = [ phase_group[ phase_group['Instance Number'].apply(
                lambda x: j*phase_group.shape[0]//3 < x and x <= (j+1)*phase_group.shape[0]//3) ] 
                     for j in range(3) ]
            file_path['P'].add(extract_unique_value(series_group, 'FilePath_4'))
            file_path['P_files'].append([ list(phase[j]['FilePath'].values) for j in range(3) ])

    file_path['M'] = list(file_path['M']) 
    file_path['P'] = list(file_path['P']) 
    file_path['root'] = extract_unique_value(group, 'FilePath_3').rsplit(':', maxsplit=1)[-1]

    num_rows = extract_unique_value(group, 'Rows')
    num_cols = extract_unique_value(group, 'Columns')
    num_slice_locations = len(group['Slice Location'].unique()) # FIXME: sanity check!
    cardiac_number_of_images = extract_unique_value(group, 'Cardiac Number of Images')
    
    assert magnitude.shape[0] == num_slice_locations*cardiac_number_of_images
    for j in range(3):
        assert phase[j].shape[0] == num_slice_locations*cardiac_number_of_images

    orientation = extract_unique_value(group, 'Image Orientation (Patient)')
    row_unit = np.array(orientation[:3])
    col_unit = np.array(orientation[3:])
    ort_unit = np.cross(col_unit, row_unit)
    assert (np.linalg.norm(ort_unit) - 1.) < 1e-12

    magnitude_positions = magnitude['Image Position (Patient)'].apply(lambda x: np.array(x))    
    magnitude_position_origin = np.array(magnitude.loc[magnitude['Instance Number'].idxmin(),'Image Position (Patient)'])    
    magnitude_slice_position_shift = magnitude_positions.apply(lambda x: np.dot(ort_unit,x - magnitude_position_origin)) - magnitude['Slice Location']    
    assert (magnitude_slice_position_shift - magnitude_slice_position_shift.mean()).abs().max() < 1e-11
    
    slice_locations = magnitude['Slice Location'][:num_slice_locations].values # FIXME: sanity check!
    assert len(np.unique(slice_locations)) == num_slice_locations
#     magnitude_slice_locations = magnitude['Slice Location'][:num_slice_locations].values    
    magnitude_trigger_times = magnitude['Trigger Time'][::num_slice_locations].values
    assert len(np.unique(magnitude_trigger_times)) == cardiac_number_of_images
    magnitude_values = np.ndarray(shape=(num_rows, num_cols, num_slice_locations, cardiac_number_of_images))
    # TODO: Check correctness of pixel value reading
    for t,_ in enumerate(magnitude_trigger_times):
        for z,_ in enumerate(slice_locations): # magnitude_slice_locations
            img_row = magnitude[magnitude['Instance Number'] == t*num_slice_locations + z + 1]; assert len(img_row) == 1; img_row = img_row.iloc[0]
            dcm_img = dcmread(img_row['FilePath'])
            magnitude_values[:,:,z,t] = dcm_img.pixel_array # no rescale properties
   

    phase_positions = []
    phase_slice_locations = []
    phase_trigger_times = []
    phase_values = np.ndarray(shape=(num_rows, num_cols, num_slice_locations, cardiac_number_of_images, 3))
    for j in range(3):
        # If this does not work can validate the phase z-coordinates here as well
        assert (slice_locations == phase[j]['Slice Location'][:num_slice_locations].values).all()
        
        venc_j = venc['venc_xy'] if j < 2 else venc['venc_z']

#         phase_slice_locations.append( phase[j]['Slice Location'][:num_slice_locations].values )    
        phase_trigger_times.append( phase[j]['Trigger Time'][::num_slice_locations].values )
        print("Trigger time offsets of phase[{}] relative to magnitude in {} ms".format(j, np.unique(phase_trigger_times[j]-magnitude_trigger_times)), flush=True)
        for t,_ in enumerate(phase_trigger_times[j]):
            for z,_ in enumerate(slice_locations): # phase_slice_locations[j]
                img_row = phase[j][phase[j]['Instance Number'] == j*cardiac_number_of_images*num_slice_locations + t*num_slice_locations + z + 1]; assert len(img_row) == 1; img_row = img_row.iloc[0]
                dcm_img = dcmread(img_row['FilePath'])
                phase_values[:,:,z,t,j] = venc_j/np.abs(dcm_img.RescaleIntercept)*(dcm_img.RescaleSlope*dcm_img.pixel_array+dcm_img.RescaleIntercept)

    print("Inverting y-phases as visually observed to be along wrong direction in samples (e.g. 10).")
    phase_values[:,:,:,:,1] = -phase_values[:,:,:,:,1]
    
    #check for uniqueness!
    pixel_spacing = extract_unique_value(group, 'Pixel Spacing')
    geometry = [pixel_spacing[0]*(np.arange(0,num_rows)+0.5),
                pixel_spacing[1]*(np.arange(0,num_cols)+0.5),
                slice_locations]

    #check for uniqueness!
    #group['Nominal Interval'].unique()  #.apply(lambda x: agg_unique(x))
    heart_cycle_period = group['Nominal Interval'].mean()
    
    print("Writing {} to HDF5...".format(name))
    flow_mri_metainfo = {
        "file_path": file_path,
        "venc": venc,
    }

    flow_mris.append({
        "study_instance_uid": name[3],
        "sequence_name": name[4],
        "cardiac_number_of_images": cardiac_number_of_images,
        "num_slice_locations": num_slice_locations,
        "heart_cycle_period": heart_cycle_period,
        "geometry": geometry,
        "magnitude_trigger_times": magnitude_trigger_times,
        "magnitude_values": magnitude_values, #"phase_slice_locations": phase_slice_locations,
        "phase_trigger_times": phase_trigger_times,
        "phase_values": phase_values,
    })
    flow_mris[-1].update(flow_mri_metainfo)
    
    hpc_predict_mri = FlowMRI(geometry=flow_mris[-1]['geometry'], 
                          time=flow_mris[-1]['magnitude_trigger_times'], # FIXME: phase times!
                          time_heart_cycle_period=flow_mris[-1]['heart_cycle_period'], 
                          intensity=flow_mris[-1]['magnitude_values'], 
                          velocity_mean=flow_mris[-1]['phase_values'], 
                          velocity_cov=None) # np.zeros(shape=phase_values.shape+(3,))) # FIXME: waste of memory space
    filename_base = os.path.join(output_root, str(sample) , file_path['root'].replace('/','-'))
    hpc_predict_mri.write_hdf5(filename_base + '.h5')
    with open(filename_base + '.json', 'w') as flow_mri_metainfo_f:
        json.dump(flow_mri_metainfo, flow_mri_metainfo_f)

In [ ]:
hpc_predict_mri = FlowMRI.read_hdf5(filename_base + '.h5')

In [ ]:
import matplotlib.pyplot as plt

figure, ax = plt.subplots(ncols=1, figsize=(12*1,8*1))
ax.hist(flow_mris[-1]['magnitude_values'].flatten(), bins=50, label='magnitude', hatch='*')
ax.hist(hpc_predict_mri.intensity.flatten(), bins=50, label='magnitude-hpc-predict', alpha=0.5)
figure.legend()
figure.tight_layout()

In [ ]:
figure, ax = plt.subplots(ncols=3, sharex=True, figsize=(12*3,8*1))
for j in range(3):
    ax[j].hist(flow_mris[0]['phase_values'][:,:,:,:,j].flatten(), bins=50, label='phase-{}'.format(j), hatch='*')
    ax[j].hist(hpc_predict_mri.velocity_mean[:,:,:,:,j].flatten(), bins=50, label='phase-{}-hpc-predict'.format(j), alpha=0.5)
figure.legend()
figure.tight_layout()